In [1]:
import re
import time
from config import username, password
from splinter import Browser
from Models.DB import DB

In [10]:
db = DB()
db.connect()
db.createTableReviews("glassdoor")

You are connected to -  ['PostgreSQL 11.5 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.3 20140911 (Red Hat 4.8.3-9), 64-bit'] 

(['companyreviews', 'public', 'reviews', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None], ['companyreviews', 'public', 'responses', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None], ['companyreviews', 'public', 'glassdoor_reviews', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None])


In [3]:
#db.dropTables("glassdoor_reviews")

(['companyreviews', 'public', 'reviews', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None], ['companyreviews', 'public', 'responses', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None])


In [4]:
urls = ["https://www.glassdoor.com/index.htm","https://www.glassdoor.com/Reviews/Daugherty-Business-Solutions-Reviews-E233317.htm"]

In [5]:
def login(url=urls[0]):
    """
        Deprecated
    """
    global browser
    
    executable_path = {'executable_path': 'chromedriver.exe'}
    browser = Browser('chrome', **executable_path, headless=False)
    browser.driver.set_window_size(1280, 720)
    
    browser.visit(url)
    
    button = browser.links.find_by_partial_href("/profile/login_input.htm?userOriginHook=HEADER_SIGNIN_LINK")
    
    browser.visit( button._element.get_attribute("href"))
    
    browser.fill('username', username)
    browser.fill('password', password)
    button = browser.find_by_css("button[class='gd-ui-button minWidthBtn css-1sdotxz']")
    button.click()

In [6]:
def visitURL(url=urls[1]):
    global browser
    
    executable_path = {'executable_path': 'chromedriver.exe'}
    browser = Browser('chrome', **executable_path, headless=False)
    browser.driver.set_window_size(1280, 720)
    
    browser.visit(url)

In [7]:
visitURL(url=urls[1])
# Must manually sign on after visiting.

In [8]:
def scrapeReviews():
    """
        header, body, rating, role, date, response, source
    """
    
    _id = 0
    
    while browser.is_element_present_by_css("a[class='pagination__ArrowStyle__nextArrow  ']"):
        
        reviews = browser.find_by_css("div[class='hreview']")
        button = browser.find_by_css("a[class='pagination__ArrowStyle__nextArrow  ']")
        
        for review in reviews:

            _id += 1
            
            header = review.find_by_css("a[class='reviewLink']").text.replace("%", "%%").replace("$", "\$")
            body = review.find_by_css("div[class='col-sm-11 pl-sm-lg mx-0']").text.replace("%", "%%").replace("$", "\$")
            rating = float(review.find_by_css('span[class="value-title"]')._element.get_attribute('title'))
            role = review.find_by_css('span[class="authorJobTitle middle reviewer"]').text.replace("%", "%%").replace("$", "\$")
            try:
                date = review.find_by_css("time[class='date subtle small']").text
            except:
                date = "0"
            source = "glassdoor"
            
            try:
                response = review.find_by_css('div[class="comment empRepComment col-12 pt-lg"]').text.replace("%", "%%").replace("$", "\$")
                
            except Exception as e:
                
                response = "No response"
                
                print(f"Exception: {e}\n")
            
            print(f"Id: {_id}\nDate: {date}\n{header}\n{rating}\n{body}\n{source}\nResponse: {response}\n-----------------------------------------------------------------------------")
            
            db.intoReviews("glassdoor", header, body, rating, role, date, response, source)
            
        #simulates clicking "next" button
        browser.visit( button._element.get_attribute("href"))

In [11]:
scrapeReviews()

Exception: 'ElementList' object has no attribute 'text'

Id: 1
Date: 0
"Invests in its employees and you can tell"
5.0
"Invests in its employees and you can tell"
Current Employee - Consultant in Saint Louis, MO
Recommends
Positive Outlook
Approves of CEO
I have been working at Daugherty Business Solutions full-time for more than 8 years
Pros
- Very supportive of its employees. Daugherty continuously invests in its employees. The ICAgile partnership catapulted my career. Unmatched training program!
- Great benefits that make it as easy as possible to work while raising a family. Daugherty leadership makes this a priority each year and you can tell. 
- Really appreciate the Health &amp; Wellness program, and the free screenings, fitness center, and...
Show More
Cons
- Some growing pains as we climbed over 1,200 employees and highest revenues ever but I think employees understand there’s a balance when you have an entrepreneurial culture.
Advice to Management
- Continue to invest in curr

Id: 9
Date: December 25, 2019
"Staff Aug Firm with no vision"
1.0
"Staff Aug Firm with no vision"
Former Employee - Peincipal in Atlanta, GA
Doesn't Recommend
Negative Outlook
Disapproves of CEO
I worked at Daugherty Business Solutions full-time for more than 5 years
Pros
Good for close to retirement years
Cons
No vision, No stable leadership, No Career Path,
Advice to Management
Have professionals run this company
Helpful
Daugherty Business Solutions Response
December 27, 2019 – Marketing &amp; Communications
Hi, this is Rick Mayhall, Managing Director of Daugherty’s Atlanta office.  Last week, the Atlanta Journal Constitution confirmed us once again as one of Atlanta’s Top Work Places 2020, a distinction we’ve earned in 7 of the last 8 years.  Having assumed the Managing Director role ...
Show More
glassdoor
Response: Daugherty Business Solutions Response
December 27, 2019 – Marketing &amp; Communications
Hi, this is Rick Mayhall, Managing Director of Daugherty’s Atlanta office.  Las

Exception: 'ElementList' object has no attribute 'text'

Id: 17
Date: October 11, 2019
"Toxic Leadership Environment"
1.0
"Toxic Leadership Environment"
Former Employee - Senior Manager 
I worked at Daugherty Business Solutions full-time
Pros
Consultant delivery team is skilled.
Cons
Toxic leadership environment. Emotionally abusive.
Helpful (2)
glassdoor
Response: No response
-----------------------------------------------------------------------------
Exception: 'ElementList' object has no attribute 'text'

Id: 18
Date: September 12, 2019
"Good pay poor leadership"
1.0
"Good pay poor leadership"
Former Employee - Senior Consultant in Dallas, TX
Doesn't Recommend
Negative Outlook
Disapproves of CEO
I worked at Daugherty Business Solutions full-time for more than 3 years
Pros
-Above average salary
-Nice healthcare benefits
Cons
-Exceptionally poor leadership in Dallas office. 
-Zero fitment of projects Vs skills for technical and non technical consultants
-Stagnant growth in Dallas reg

Exception: 'ElementList' object has no attribute 'text'

Id: 27
Date: May 7, 2019
"Such a wonderful company culture!"
5.0
"Such a wonderful company culture!"
Current Employee - Digital Marketing Specialist in Saint Louis, MO
Recommends
Positive Outlook
Approves of CEO
I have been working at Daugherty Business Solutions full-time for less than a year
Pros
- New employee on-boarding process 
- Supportive and welcoming employees
- Company swag
- Helpful HR team
- Fun company culture
- Open to new and innovative internal processes 
- Off-site company-supported events
- Invest heavily in employee training and continued learning
Cons
I haven't found any cons as of yet to share!
Advice to Management
Continue investing in your new employee on-boarding process! It's been the best I've experienced yet!
Helpful
glassdoor
Response: No response
-----------------------------------------------------------------------------
Exception: 'ElementList' object has no attribute 'text'

Id: 28
Date: April 30

Exception: 'ElementList' object has no attribute 'text'

Id: 35
Date: April 11, 2019
"Part of something special here"
5.0
"Part of something special here"
Current Employee - Manager in Saint Louis, MO
Recommends
Positive Outlook
Approves of CEO
I have been working at Daugherty Business Solutions full-time for more than 10 years
Pros
The collaborative, supportive, team culture is simply amazing. Being a Daugherty employee is something to be proud of and knowing you are surrounded by people who genuinely care about you and you become part of the Team Daugherty family.
Cons
Like any family, Daugherty's here for the long-haul, so characteristics for lifelong relationships do well here - smart, practical, patience, growth, resilience, learning, purpose.
Advice to Management
Future looks even brighter with recent commitment and investments
Helpful (1)
glassdoor
Response: No response
-----------------------------------------------------------------------------
Exception: 'ElementList' object 

...

Exception: 'ElementList' object has no attribute 'text'

Id: 267
Date: January 2, 2013
"Terrible - Avoid at all costs"
1.0
"Terrible - Avoid at all costs"
Former Employee - Consultant in Chicago, IL
Doesn't Recommend
Positive Outlook
Disapproves of CEO
I worked at Daugherty Business Solutions full-time
Pros
A very good health insurance plan
Competitive Salary
Cons
No career path
Dishonest, immoral management that does not care about peoples lives.
No vision, or desire for project work, just mindless staff aug.
Advice to Management
Start treating employees with respect, because one day, maybe those employees you treated so poorly will be "decision makers" in an account you want to pursue. And the way you treated them will come home to roost.
Helpful (8)
glassdoor
Response: No response
-----------------------------------------------------------------------------
Exception: 'ElementList' object has no attribute 'text'

Id: 268
Date: November 30, 2012
"Overall Lack of Vision"
2.0
"Overall 

Exception: 'ElementList' object has no attribute 'text'

Id: 275
Date: March 5, 2012
"A high-end staffing firm that can get you experience at some of the larger companies in St. Louis."
2.0
"A high-end staffing firm that can get you experience at some of the larger companies in St. Louis."
 in Saint Louis, MO
Doesn't Recommend
Disapproves of CEO
I have been working at Daugherty Business Solutions
Pros
They are connected to the larger St. Louis Companies like Express Scripts, InBev - AB, Charter, MasterCard, Monsanto - so you can get placed at these places for experience. If you are looking for placement with benefits this is a good option.
Cons
The recruiters sold me on overtime pay, quick raise increases and promotions, reimbursement for things used at work - none of which ever happened, where recognized as a problem by management and not dealt with. They are very supportive to get you to take on more responsibilities and get the job done, then put you down as incompetent during your 

Exception: 'ElementList' object has no attribute 'text'

Id: 284
Date: May 18, 2011
"Very Disappointing"
2.0
"Very Disappointing"
 in Chicago, IL
Doesn't Recommend
No opinion of CEO
I worked at Daugherty Business Solutions
Pros
- Local Model / No Travel
- Decent middle management talent (until they decide to leave)
- Competitive Compensation
- A ton of client potential (until they screw that up)
Cons
- No New Sales
- Frequent turnover in employees
- Skill Talent is not as strong as it could be
- Poor Management Communication
- Staff Aug company trying to be something they are not
- No Career Path
Advice to Management
Learn how to retain good talent. Chicago is going nowhere with the existing model.
Helpful (11)
glassdoor
Response: No response
-----------------------------------------------------------------------------
Exception: 'ElementList' object has no attribute 'text'

Id: 285
Date: May 17, 2011
"It's what you put in to it..."
4.0
"It's what you put in to it..."
 in Saint Louis, 

Exception: 'ElementList' object has no attribute 'text'

Id: 293
Date: July 3, 2010
"Great place for local travel only consulting"
5.0
"Great place for local travel only consulting"
Senior Consultant in Chicago, IL
Recommends
No opinion of CEO
I worked at Daugherty Business Solutions
Pros
Highly experienced, well-rounded employees. Senior leadership is reasonable and seems to really want happy employees. Very fair salary and benefits. Vacation days and even sick days carry over year to year. Overtime hours are paid.
Cons
Like any consulting company, Daugherty only has so much control over the exact type of work they can put you on. They have do fill the positions clients need, so some flexibility is necessary from consultants. This is true at every consulting company, at least at Daugherty I got the sense they were doing their sincere best to make a good fit between positions and skills.
Advice to Management
I don't think I'd have any useful information to give. I think Daugherty is ru

Exception: 'ElementList' object has no attribute 'text'

Id: 301
Date: August 24, 2008
"Great people to work with if you don't mind being in the dark about what's going on."
3.0
"Great people to work with if you don't mind being in the dark about what's going on."
Principal in Saint Louis, MO
I worked at Daugherty Business Solutions
Pros
The people you work with are extremely good people - one thing Daugherty is generally very good at is recruiting great people at the consulting level - not so much at the management level.
Cons
Total lack of communication from management - management is always behind closed doors and rarely communicates anything to the consulting staff. Secondly, the company has a tough time growing beyond it's roots as a staff augmentation company - there is no effort to build and sustain organizational competency like a true consulting organization needs to do to grow beyond what the owner can get his own arms around. ...
Show More
Advice to Management
Communicate, c